In [25]:
from pyspark.sql import SparkSession

appName = "PySpark Example"
master = "local"
# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.jars", "/usr/local/spark/jars/mysql-connector-java-8.0.30.jar") \
    .getOrCreate()



In [26]:

server = "localhost"
port = 3306
database = "nation"
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"


user = "alinizar"
password = "alinizarpwd"
jdbc_driver = "com.mysql.cj.jdbc.Driver"


properties = {
    "user": user,
    "password": password,
    "driver": jdbc_driver
}


In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [28]:
continents_df = spark.read.jdbc(jdbc_url, "(select * from continents) tab", properties=properties)
countries_df = spark.read.jdbc(jdbc_url, "(select * from countries) tab", properties=properties)
country_languages_df = spark.read.jdbc(jdbc_url, "(select * from country_languages) tab", properties=properties)
country_stats_df = spark.read.jdbc(jdbc_url, "(select * from country_stats) tab", properties=properties)
languages_df = spark.read.jdbc(jdbc_url, "(select * from languages) tab", properties=properties)
regions_df = spark.read.jdbc(jdbc_url, "(select * from regions) tab", properties=properties)

In [29]:
# 1. Group by continent and show the total area for each continent
# see the use of alias.
report1 = (
    countries_df
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(sum("area").alias("total_area"))
    .orderBy("total_area", ascending=False)
)
report1.show()

+--------------------+-----------+
|                name| total_area|
+--------------------+-----------+
|       North America|21500515.00|
|      Eastern Europe|18814094.00|
|       South America|17864926.00|
|          Antarctica|13132101.00|
|        Eastern Asia|11774482.00|
|Southern and Cent...|10791130.00|
|     Northern Africa| 8524703.00|
|Australia and New...| 8011939.00|
|      Central Africa| 6612667.00|
|      Eastern Africa| 6299891.00|
|      Western Africa| 6138338.00|
|         Middle East| 4820592.00|
|      Southeast Asia| 4494801.00|
|     Southern Africa| 2674778.00|
|     Central America| 2479532.00|
|    Nordic Countries| 1321901.00|
|     Southern Europe| 1316392.40|
|      Western Europe| 1108456.50|
|           Melanesia|  540774.00|
|     British Islands|  313173.00|
+--------------------+-----------+
only showing top 20 rows



In [30]:

# 2. Group by continent and show the average population for each continent
report2 = (
    country_stats_df
    .join(countries_df, country_stats_df.country_id == countries_df.country_id, "left")
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(avg("population").alias("avg_population"))
    .orderBy("avg_population", ascending=False)
)
report2.show()

+--------------------+--------------------+
|                name|      avg_population|
+--------------------+--------------------+
|        Eastern Asia|2.3696532803215435E8|
|Southern and Cent...|1.1626822492694806E8|
|       North America|  7.56906064587156E7|
|      Southeast Asia| 4.705655576433121E7|
|      Eastern Europe|3.1835508722222224E7|
|     British Islands| 3.091303916101695E7|
|     Northern Africa| 2.538382808280255E7|
|       South America|2.4169269097560976E7|
|      Western Europe| 1.976610894057377E7|
|     Southern Europe|1.4624393971717171E7|
|     Central America|1.4049062190578159E7|
|      Western Africa|1.3454578974850299E7|
|      Eastern Africa|1.2982107459926017E7|
|Australia and New...|1.0232338661016949E7|
|         Middle East|   9978791.629726207|
|      Central Africa|   9791947.927038627|
|     Southern Africa|   8796830.916363636|
|    Nordic Countries|   4390169.945859873|
|    Baltic Countries|   2262633.597222222|
|           Caribbean|       224

In [31]:


# 3. Count the number of countries in each region
report3 = (
    countries_df
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(count("country_id").alias("num_countries"))
    .orderBy("num_countries", ascending=False)
)
report3.show()


+--------------------+-------------+
|                name|num_countries|
+--------------------+-------------+
|           Caribbean|           24|
|      Eastern Africa|           20|
|         Middle East|           18|
|      Western Africa|           17|
|     Southern Europe|           15|
|       South America|           14|
|Southern and Cent...|           14|
|      Southeast Asia|           11|
|      Eastern Europe|           10|
|           Polynesia|           10|
|      Western Europe|            9|
|      Central Africa|            9|
|     Central America|            8|
|        Eastern Asia|            8|
|     Northern Africa|            7|
|    Nordic Countries|            7|
|          Micronesia|            7|
|Australia and New...|            5|
|       North America|            5|
|          Antarctica|            5|
+--------------------+-------------+
only showing top 20 rows



In [32]:

# 4. Show the top 5 languages spoken worldwide and the number of countries where they are official
report4 = (
    country_languages_df
    .join(languages_df.alias("languages"), country_languages_df.language_id == languages_df.language_id, "left")
    .groupBy("languages.language")
    .agg(count("country_id").alias("num_countries"))
    .orderBy("num_countries", ascending=False)
    .limit(5)
)
report4.show()

+--------+-------------+
|language|num_countries|
+--------+-------------+
| English|           60|
|  Arabic|           33|
| Spanish|           28|
|  French|           25|
| Chinese|           19|
+--------+-------------+



In [33]:

# 5. Show the GDP per capita for each country in 2022
report5 = (
    country_stats_df
    .filter(col("year") == 2000)
    .join(countries_df.alias("countries"), country_stats_df.country_id == countries_df.country_id, "left")
    .withColumn("gdp_per_capita", col("gdp") / col("population"))
    .select("countries.name", "gdp_per_capita")
    .orderBy("gdp_per_capita", ascending=False)
)
report5.show()

+--------------------+-----------------+
|                name|   gdp_per_capita|
+--------------------+-----------------+
|              Monaco|82367.99129001151|
|       Liechtenstein|74853.93873553520|
|             Bermuda|56284.16864780942|
|          Luxembourg|48735.99549163420|
|          San Marino|40124.45990823684|
|               Japan|38532.04354990027|
|              Norway|38146.79555650264|
|         Switzerland|37868.25347113477|
|       United States|36334.74764999793|
|United Arab Emirates|33291.30055499859|
|             Iceland|32018.06324567486|
|             Denmark|30743.59654327203|
|               Qatar|29976.11703923250|
|              Sweden|29323.24208370298|
|      United Kingdom|27987.76768130496|
|             Bahamas|27097.61081157252|
|             Ireland|26241.36461460107|
|         Netherlands|26149.42451147414|
|           Hong Kong|25756.63915978995|
|             Austria|24564.48589451800|
+--------------------+-----------------+
only showing top

In [34]:


# 6. Show the total number of countries for each continent

report6 = (
    countries_df.alias("countries")
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(countDistinct("countries.country_id").alias("num_countries"))
)
report6.show()

+--------------------+-------------+
|                name|num_countries|
+--------------------+-------------+
|      Eastern Europe|           10|
|     Southern Europe|           15|
|     British Islands|            2|
|     Central America|            8|
|Micronesia/Caribbean|            1|
|Australia and New...|            5|
|       North America|            5|
|           Caribbean|           24|
|      Western Africa|           17|
|      Eastern Africa|           20|
|          Antarctica|            5|
|       South America|           14|
|           Polynesia|           10|
|     Northern Africa|            7|
|           Melanesia|            5|
|         Middle East|           18|
|     Southern Africa|            5|
|      Western Europe|            9|
|Southern and Cent...|           14|
|    Nordic Countries|            7|
+--------------------+-------------+
only showing top 20 rows



In [35]:

# 7. Show the average GDP and population for each region in 2000
report7 = (
    country_stats_df
    .filter(col("year") == 2000)
    .join(countries_df, country_stats_df.country_id == countries_df.country_id, "left")
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(avg("gdp").alias("avg_gdp"), avg("population").alias("avg_population"))
)
report7.show()

+--------------------+------------------+--------------------+
|                name|           avg_gdp|      avg_population|
+--------------------+------------------+--------------------+
|      Eastern Europe|  69790091382.2222|3.1297240222222224E7|
|     Southern Europe| 170614424145.6667|1.1201738833333334E7|
|     British Islands| 874061479178.0000|         3.1348844E7|
|     Central America|  97409661724.7500|       1.691529375E7|
|Australia and New...| 233923140978.5000|          1.150535E7|
|       North America|2749785812457.5000|        7.82415435E7|
|           Caribbean|  10272337187.6000|  2467434.7333333334|
|      Eastern Africa|   4438484140.0000|      1.5087283625E7|
|      Western Africa|   6875190354.5625|     1.46714108125E7|
|       South America| 112193603067.8333|        2.90204855E7|
|           Polynesia|    983167099.2500|           130626.75|
|     Northern Africa|  44247809175.8333|2.8501455166666668E7|
|           Melanesia|   1718983010.4000|           149

In [36]:

# 8. Show the top 5 regions with the highest total GDP in 2000
report8 = (
    country_stats_df
    .filter(col("year") == 2000)
    .join(countries_df, country_stats_df.country_id == countries_df.country_id, "left")
    .join(regions_df.alias("regions"), countries_df.region_id == regions_df.region_id, "left")
    .groupBy("regions.name")
    .agg(sum("gdp").alias("total_gdp"))
    .orderBy("total_gdp", ascending=False)
    .limit(5)
)
report8.show()

+---------------+--------------+
|           name|     total_gdp|
+---------------+--------------+
|  North America|10999143249830|
|   Eastern Asia| 6840028388530|
| Western Europe| 4461798351752|
|Southern Europe| 2047373089748|
|British Islands| 1748122958356|
+---------------+--------------+

